In [81]:
import numpy as np
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [96]:
dataset = pd.read_csv('tweets-total-csv-3-class-balanced.csv', encoding='ISO-8859-1', delimiter=';')

In [97]:
tweets = dataset['tweet'].values
classes = dataset['OPINIAO'].values

In [98]:
def Preprocessamento(instancia):
    #remove links, pontos, virgulas,ponto e virgulas dos tweets
    #coloca tudo em minusculo
    fraseslimpas = []
    for palavras in instancia:
        palavras = re.sub(r"http\S+", "", palavras).lower().replace(',','').replace('.','').replace(';','').replace('-','').replace(':','').replace('?','')
        fraseslimpas.append(palavras)
    return (fraseslimpas)

In [99]:
def RemoveStopWords(instancia):
    #remove stopwords
    fraseslimpas = []
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')
    for palavras in instancia:
        semstop = [p for p in palavras.split() if p not in stopwords]
        fraseslimpas.append((str(semstop)))
    return (fraseslimpas)

In [100]:
def Steaming(instancia):
    #remove stopwords
    fraseslimpas = []
    stemmer = nltk.stem.RSLPStemmer()
    for palavras in instancia:
        steam = [str(stemmer.stem(p)) for p in palavras.split()]
        fraseslimpas.append((str(steam)))
    return (fraseslimpas)

In [101]:
#tweetsprocessadossteaming = Steaming(tweets)
tweetsprocessados = Preprocessamento(tweets)
removestopwords = RemoveStopWords(tweetsprocessados)

In [102]:
print(tweets[1])
print(tweetsprocessados[1])
print(removestopwords[1])

O #BancoDoBrasil já me deixou uma mensagem de parabéns. O 1º.
o #bancodobrasil já me deixou uma mensagem de parabéns o 1º
['#bancodobrasil', 'deixou', 'mensagem', 'parabéns', '1º']


In [103]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(removestopwords)
modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [104]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=50)

print('Porcentagem de acerto: %f ' % (metrics.accuracy_score(classes, resultados)*100))

Porcentagem de acerto: 76.706827 


In [105]:
opiniao=['Positivo', 'Negativo', 'Neutro']
print(metrics.classification_report(classes,resultados,opiniao))

              precision    recall  f1-score   support

    Positivo       0.68      0.64      0.66       166
    Negativo       0.93      0.98      0.96       166
      Neutro       0.68      0.68      0.68       166

    accuracy                           0.77       498
   macro avg       0.76      0.77      0.76       498
weighted avg       0.76      0.77      0.76       498



In [106]:
dataset2 = pd.read_csv('Dados3.csv', encoding='ISO-8859-1', delimiter=';')

In [107]:
tweets = dataset2['Text'].values
classes = dataset2['Opniao'].values

In [108]:
tweetsprocessados = Preprocessamento(tweets)
removestopwords = RemoveStopWords(tweetsprocessados)

In [109]:
print(tweetsprocessados[1])
print(removestopwords[1])

@senadorhumberto duas merdas pt e folha de são paulo  

em breve não existirá nenhum 
['@senadorhumberto', 'duas', 'merdas', 'pt', 'folha', 'paulo', 'breve', 'não', 'existirá', 'nenhum']


In [110]:
freq_tweets = vectorizer.transform(removestopwords)

In [111]:
opniao = modelo.predict(freq_tweets)

In [112]:
print('Porcentagem de acerto: %f ' % (metrics.accuracy_score(classes, opniao)*100))

Porcentagem de acerto: 46.428571 


In [113]:
tipo=['Positivo', 'Negativo', 'Neutro']
print(metrics.classification_report(classes,opniao,tipo))

              precision    recall  f1-score   support

    Positivo       0.17      0.25      0.20         4
    Negativo       0.67      0.43      0.52        14
      Neutro       0.46      0.60      0.52        10

    accuracy                           0.46        28
   macro avg       0.43      0.43      0.41        28
weighted avg       0.52      0.46      0.48        28



In [114]:
from nltk.metrics import ConfusionMatrix
esperado = []
previsto = []
for opniao in opniao:
    previsto.append(opniao)
for classe in classes:
    esperado.append(classe)    

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

         | N   P |
         | e   o |
         | g N s |
         | a e i |
         | t u t |
         | i t i |
         | v r v |
         | o o o |
---------+-------+
Negativo |<6>4 4 |
  Neutro | 3<6>1 |
Positivo | . 3<1>|
---------+-------+
(row = reference; col = test)



In [115]:
def BuscaOpiniao(frase):
    tweetsprocessados = Preprocessamento(frase)
    removestopwords = RemoveStopWords(tweetsprocessados)
    freq_tweets = vectorizer.transform(removestopwords)
    opniao = modelo.predict(freq_tweets)
    prob = modelo.predict_proba(freq_tweets).max()
    return opniao,prob

In [116]:
dataset3 = pd.read_csv('Dados2.csv', encoding='ISO-8859-1', delimiter=';')
datasetClassificado = pd.DataFrame(columns=['Text','Opiniao','Probabilidade'])

In [117]:
for text in dataset3['Text']:
    global opiniao
    opiniao = BuscaOpiniao([text])
    datasetClassificado = datasetClassificado.append({'Text':text,'Opiniao':opiniao[0][0],'Probabilidade':opiniao[1]}, ignore_index=True)

In [118]:
datasetClassificado.to_excel('DadosClassificadosB.xlsx')